In [1]:
!pip install groq python-dotenv numpy tqdm datasets

  Using cached anyio-4.9.0-py3-none-any.whl.metadata (4.7 kB)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached pydantic-2.11.7-py3-none-any.whl.metadata (67 kB)
  Using cached httpcore-1.0.9-py3-none-any.whl.metadata (21 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached pydantic_core-2.33.2-cp310-cp310-macosx_11_0_arm64.whl.metadata (6.8 kB)
  Using cached typing_inspection-0.4.1-py3-none-any.whl.metadata (2.6 kB)
Using cached anyio-4.9.0-py3-none-any.whl (100 kB)
Using cached distro-1.9.0-py3-none-any.whl (20 kB)
Using cached httpx-0.28.1-py3-none-any.whl (73 kB)
Using cached httpcore-1.0.9-py3-none-any.whl (78 kB)
Using cached pydantic-2.11.7-py3-none-any.whl (444 kB)
Using cached pydantic_core-2.33.2-cp310-cp310-macosx_11_0_arm64.whl (1.9 MB)
Using cached annotated_types-0.7.0-py3-none-any.whl (13 kB)
Using cached typing_inspection-0.4.1-py3-none-any.wh

In [3]:
from groq import Groq
from dotenv import load_dotenv
from datasets import load_dataset

import os
from tqdm import tqdm
import re
import random
import pprint

from typing import List, Dict, Any

load_dotenv()
random.seed(0)

client = Groq()
gsm8k_dataset = load_dataset("gsm8k", "main")

gsm8k_train = gsm8k_dataset["train"]
gsm8k_test  = gsm8k_dataset["test"]

Generating test split: 100%|██████████| 1319/1319 [00:00<00:00, 391527.74 examples/s]


In [5]:
def generate_response_using_Llama(
        prompt: str,
        model: str = "llama3-8b-8192"
    ):
    try:
        chat_completion = client.chat.completions.create(
            messages=[
                {
                    "role": "system",
                    "content": "You are a helpful assistant that solves math problems."
                },
                {
                    "role": "user", 
                    "content": prompt
                }
            ],
            model=model,
            temperature=0.3, ### 수정해도 됩니다!
            stream=False
        )
        return chat_completion.choices[0].message.content
    
    except Exception as e:
        print(f"API call error: {str(e)}")
        return None

#### 응답 잘 나오는지 확인해보기

In [6]:
response = generate_response_using_Llama(
    prompt="Hello world!",
)
print(response)

Hello! I'm excited to help you with any math problems you have. What kind of math are you working on? Do you have a specific problem you're stuck on or a concept you're struggling with?


#### GSM8K 데이터셋 확인해보기

In [8]:
print("[Question]")
for l in gsm8k_test['question'][0].split("."):
    print(l)
print("="*100)
print("[Answer]")
print(gsm8k_test['answer'][0])

[Question]
Janet’s ducks lay 16 eggs per day
 She eats three for breakfast every morning and bakes muffins for her friends every day with four
 She sells the remainder at the farmers' market daily for $2 per fresh duck egg
 How much in dollars does she make every day at the farmers' market?
[Answer]
Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eggs a day.
She makes 9 * 2 = $<<9*2=18>>18 every day at the farmer’s market.
#### 18


#### Util 함수들
- extract_final_answer: LLM의 응답을 parse하여 최종 결과만 추출 (정답과 비교하기 위해)
- run_benchmark_test: 벤치마크 테스트
- save_final_result: 결과물 제출을 위한 함수

In [9]:
### 수정해도 됩니다!
def extract_final_answer(response: str):
    regex = r"(?:Answer:|Model response:)\s*\$?([0-9,]+)\b|([0-9,]+)\s*(meters|cups|miles|minutes)"
    matches = re.finditer(regex, response, re.MULTILINE)
    results = [match.group(1) if match.group(1) else match.group(2).replace(",", "") for match in matches]

    if len(results) == 0:
        additional_regex = r"\$?([0-9,]+)"
        additional_matches = re.finditer(additional_regex, response, re.MULTILINE)
        results.extend([match.group(1).replace(",", "") for match in additional_matches])

    return results[-1] if results else None

In [ ]:
### 수정해도 됩니다!
def run_benchmark_test(
        dataset,
        prompt: str,
        model: str = "llama3-8b-8192",
        num_samples: int = 50,
        VERBOSE: bool = False
    ):
    correct = 0
    total   = 0
    results = []

    for i in tqdm(range(min(num_samples, len(dataset)))):
        question = dataset[i]["question"]
        correct_answer = float(re.findall(r'\d+(?:\.\d+)?', dataset[i]["answer"].split('####')[-1])[0])

        response = generate_response_using_Llama(
            prompt=prompt.format(question=question),
            model=model
        )

        if response:
            if VERBOSE:
                print("="*50)
                print(response)
                print("="*50)
            predicted_answer = extract_final_answer(response)

            if isinstance(predicted_answer, str):
                predicted_answer = float(predicted_answer.replace(",", ""))
            
            diff = abs(predicted_answer - correct_answer)
            is_correct = diff < 1e-5 if predicted_answer is not None else False
            
            if is_correct:
                correct += 1
            total += 1
            
            results.append({
                'question': question,
                'correct_answer': correct_answer,
                'predicted_answer': predicted_answer,
                'response': response,
                'correct': is_correct
            })

            if (i + 1) % 5 == 0:
                current_acc = correct/total if total > 0 else 0
                print(f"Progress: [{i+1}/{num_samples}]")
                print(f"Current Acc.: [{current_acc:.2%}]")

    return results, correct/total if total > 0 else 0

In [11]:
def save_final_result(results: List[Dict[str, Any]], accuracy: float, filename: str) -> None:
    result_str = f"====== ACCURACY: {accuracy} ======\n\n"
    result_str += f"[Details]\n"
    
    for idx, result in enumerate(results):
        result_str += f"Question {idx+1}: {result['question']}\n"
        result_str += f"Correct Answer: {result['correct_answer']}\n"
        result_str += f"Predicted Answer: {result['predicted_answer']}\n"
        result_str += f"Correct: {result['correct']}\n\n"
    
    with open(filename, "w", encoding="utf-8") as f:
        f.write(result_str)

#### Direct prompting with few-shot example

In [20]:
def construct_direct_prompt(num_examples: int = 0) -> str:
    train_dataset = gsm8k_train

    sampled_indices = random.sample(
        [i for i in range(len(train_dataset['question']))],
        num_examples
    )

    prompt = "Instruction:\nSolve the following mathematical question and generate ONLY the answer after a tag, 'Answer:' without any rationale.\n"

    for i in range(num_examples):
        cur_question = train_dataset['question'][i]
        cur_answer = train_dataset['answer'][i].split("####")[-1].strip()

        prompt += f"\n[Example {i+1}]\n"
        prompt += f"Question:\n{cur_question}\n"
        prompt += f"Answer:{cur_answer}\n"

    prompt += "\nQuestion:\n{question}\nAnswer:"

    return prompt

In [ ]:
### 어떤 방식으로 저장되는지 확인해보세요!
PROMPT = construct_direct_prompt(0)
VERBOSE = False

results, accuracy = run_benchmark_test(
    dataset=gsm8k_test,
    prompt=PROMPT,
    VERBOSE=VERBOSE,
    num_samples=10
)
save_final_result(results, accuracy, "example.txt")

 50%|█████     | 5/10 [00:03<00:02,  1.79it/s]

Progress: [5/10]
Current Acc.: [40.00%]


100%|██████████| 10/10 [00:05<00:00,  1.70it/s]

Progress: [10/10]
Current Acc.: [20.00%]


In [ ]:
# TODO: 0 shot, 3 shot, 5 shot direct prompting을 통해 벤치마크 테스트를 한 후, 각각 direct_prompting_{shot: int}.txt로 저장해주세요!
# 예시: shot이 5인 경우 direct_prompting_5.txt
# 항상 num_samples=50 입니다!
# 0 shot = 0.16
# 3 shot = 0.16
# 5 shot = 0.18

shots = [0, 3, 5]

for shot in shots:
    prompt = construct_direct_prompt(num_examples=shot)
    VERBOSE = False

    results, accuracy = run_benchmark_test(
        dataset=gsm8k_test,
        prompt=prompt,
        VERBOSE=VERBOSE,
        num_samples=50
    )

    save_final_result(results, accuracy, f"direct_prompting_{shot}.txt")
    print(f"{shot}-shot done. Accuracy: {accuracy:.3f}")


 10%|█         | 5/50 [00:02<00:21,  2.09it/s]

Progress: [5/50]
Current Acc.: [40.00%]


 20%|██        | 10/50 [00:05<00:23,  1.72it/s]

Progress: [10/50]
Current Acc.: [20.00%]


 30%|███       | 15/50 [00:08<00:20,  1.73it/s]

...

Progress: [40/50]
Current Acc.: [17.50%]


 90%|█████████ | 45/50 [02:01<00:12,  2.50s/it]

Progress: [45/50]
Current Acc.: [15.56%]


100%|██████████| 50/50 [02:17<00:00,  2.75s/it]

Progress: [50/50]
Current Acc.: [18.00%]
5-shot done. Accuracy: 0.180


### Chain-of-Thought prompting with few-shot example
```text
[Question]
Janet’s ducks lay 16 eggs per day
 She eats three for breakfast every morning and bakes muffins for her friends every day with four
 She sells the remainder at the farmers' market daily for $2 per fresh duck egg
 How much in dollars does she make every day at the farmers' market?
====================================================================================================
[Answer]
Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eggs a day.
She makes 9 * 2 = $<<9*2=18>>18 every day at the farmer’s market.
#### 18
```

[Answer] 아래의 정답을 도출하는 과정을 예시로 달아주면 CoT의 few shot이 되겠죠?

In [25]:
def construct_CoT_prompt(num_examples: int = 3) -> str:
    train_dataset = gsm8k_train

    sampled_indices = random.sample(
        [i for i in range(len(train_dataset['question']))],
        num_examples
    )
    prompt = "Instruction:\nSolve the following mathematical question step-by-step. Show your reasoning procedure and then generate the final answer after the tag 'Answer:'.\n"

    for i in range(num_examples):
        #TODO: CoT example을 만들어주세요!
        current_question = train_dataset['question'][sampled_indices[i]]
        full_answer = train_dataset['answer'][sampled_indices[i]].strip()

        prompt += f"\n[Example {i+1}]\n"
        prompt += f"Question:\n{current_question}\n"
        prompt += f"{full_answer}\n"

    prompt += "\nQuestion:\n{question}\nAnswer:"

    return prompt

In [ ]:
# TODO: 0 shot, 3 shot, 5 shot CoT prompting을 통해 벤치마크 테스트를 한 후, 각각 CoT_prompting_{shot: int}.txt로 저장해주세요!
# 예시: shot이 5인 경우 CoT_prompting_5.txt
# 항상 num_samples=50 입니다!
# 0 shot : 0.720
# 3 shot : 0.680
# 5 shot : 0.700

shots = [0, 3, 5]

for shot in shots:
    prompt = construct_CoT_prompt(num_examples=shot)
    VERBOSE = False

    results, accuracy = run_benchmark_test(
        dataset=gsm8k_test,
        prompt=prompt,
        VERBOSE=VERBOSE,
        num_samples=50
    )

    save_final_result(results, accuracy, f"CoT_prompting_{shot}.txt")
    print(f"{shot}-shot done. Accuracy: {accuracy:.3f}")


 10%|█         | 5/50 [00:05<00:53,  1.18s/it]

Progress: [5/50]
Current Acc.: [80.00%]


 20%|██        | 10/50 [00:10<00:38,  1.03it/s]

Progress: [10/50]
Current Acc.: [70.00%]


 30%|███       | 15/50 [00:14<00:29,  1.20it/s]

...

Progress: [40/50]
Current Acc.: [67.50%]


 90%|█████████ | 45/50 [07:30<00:49,  9.93s/it]

Progress: [45/50]
Current Acc.: [71.11%]


100%|██████████| 50/50 [08:21<00:00, 10.03s/it]

Progress: [50/50]
Current Acc.: [70.00%]
5-shot done. Accuracy: 0.700


### Construct your prompt!!

목표: 본인만의 프롬프트를 통해 정답률을 더 끌어올려보기!
- gsm8k의 train 데이터셋에서 예시를 가져온 다음 (자유롭게!)
- 그 예시들에 대한 풀이 과정을 만들어주세요!
- 모든 것들이 자유입니다! Direct Prompting, CoT Prompting을 한 결과보다 정답률만 높으면 돼요.

In [29]:
### 자유롭게 수정해도 됩니다! 완전히 새로 함수를 만들어도 돼요.
from typing import List
def construct_my_prompt(example_list: List[str], num_examples: int = 3):
    prompt = (
        "Instruction:\n"
        "Solve each math problem by reasoning step-by-step in the scratchpad first. "
        "Then provide the final result after the tag 'Answer:'.\n"
    )
    
    for i, example in enumerate(example_list[:num_examples]):
        prompt += f"\n[Example {i+1}]\n{example.strip()}\n"

    prompt += "\nQuestion:\n{question}\nScratchpad:\nAnswer:"
    return prompt


In [ ]:
example_list = [
    """Question:
A bag has 5 red balls and 3 blue balls. If you take out 2 red balls, how many red balls remain?
Scratchpad:
Start with 5 red balls.
Take out 2 → 5 - 2 = 3 red balls left.
Answer: 3""",

    """Question:
You have 12 candies and give 4 to a friend. How many do you have left?
Scratchpad:
Start with 12 candies.
Give away 4 → 12 - 4 = 8.
Answer: 8""",

    """Question:
Tom reads 5 pages every day. How many pages does he read in 6 days?
Scratchpad:
He reads 5 pages per day.
Over 6 days: 5 * 6 = 30.
Answer: 30"""
]


In [42]:
import re

def extract_answer(text: str) -> str:
    match = re.search(r"Answer:\s*([0-9\.,\-]+)", text)
    if match:
        return match.group(1).strip()
    match2 = re.search(r"Correct Answer:\s*([^\n]+)", text)  # ← 수정
    if match2:
        return match2.group(1).strip()
    return "N/A"


In [ ]:
prompt_template = construct_my_prompt(example_list, num_examples=3)

import time

def run_with_answer_verification(prompt_template, dataset, num_samples=10):
    results = []
    correct = 0

    for i in range(num_samples):
        q = dataset['question'][i]
        gt = dataset['answer'][i].split("####")[-1].strip()

        # 1. Scratchpad prompting = reasoning + answer
        reasoning_prompt = prompt_template.format(question=q)
        reasoning_output = generate_response_using_Llama(reasoning_prompt)
        time.sleep(1.5)

        # 2. Verification prompting = reanswer
        verification_prompt = (
            "Instruction:\n"
            "Please solve the following problem again step-by-step and provide the final answer after the tag 'Answer:'. Just output the answer number.\n\n"
            f"Question:\n{q}\n\n"
            "Answer:"
        )

        verification_output = generate_response_using_Llama(verification_prompt)
        time.sleep(1.5)

        # 3. Extract final answer
        verified_answer = extract_answer(verification_output)

        try:
            is_correct = abs(float(verified_answer) - float(gt)) < 1e-3
        except:
            is_correct = False

        correct += is_correct

        results.append({
            "question": q,
            "correct_answer": gt,
            "predicted_answer": verified_answer,
            "correct": is_correct
        })

        if (i + 1) % 5 == 0:
            current_acc = correct / (i + 1)
            print(f"Progress: [{i+1}/{num_samples}]")
            print(f"Current Acc.: {current_acc:.2%}\n")


    accuracy = correct / num_samples
    return results, accuracy


In [ ]:
# TODO: 만든 0 shot, 3 shot, 5 shot example과 프롬프트를 통해 벤치마크 테스트를 한 후, 각각 My_prompting_{shot: int}.txt로 저장해주세요!
# 예시: shot이 5인 경우 My_prompting_5.txt
# 항상 num_samples=50 입니다!
# 0 shot = 0.820
# 3 shot = 0.760
# 5 shot = 0.740

shots = [0, 3, 5]

for shot in shots:
    prompt_template = construct_my_prompt(example_list, num_examples=shot)

    results, accuracy = run_with_answer_verification(
        prompt_template=prompt_template,
        dataset=gsm8k_test,
        num_samples=50
    )

    save_final_result(results, accuracy, f"My_prompting_{shot}.txt")
    print(f"{shot}-shot with verification done. Accuracy: {accuracy:.3f}")


Progress: [5/50]
Current Acc.: 80.00%

Progress: [10/50]
Current Acc.: 70.00%

Progress: [15/50]
Current Acc.: 66.67%

Progress: [20/50]
Current Acc.: 75.00%

Progress: [25/50]
Current Acc.: 76.00%

Progress: [30/50]
Current Acc.: 80.00%

Progress: [35/50]
Current Acc.: 82.86%

Progress: [40/50]
Current Acc.: 82.50%

Progress: [45/50]
Current Acc.: 82.22%

Progress: [50/50]
Current Acc.: 82.00%

0-shot with verification done. Accuracy: 0.820
Progress: [5/50]
Current Acc.: 80.00%

Progress: [10/50]
Current Acc.: 70.00%

Progress: [15/50]
Current Acc.: 66.67%

Progress: [20/50]
Current Acc.: 65.00%

Progress: [25/50]
Current Acc.: 68.00%

Progress: [30/50]
Current Acc.: 70.00%

Progress: [35/50]
Current Acc.: 74.29%

Progress: [40/50]
Current Acc.: 72.50%

Progress: [45/50]
Current Acc.: 75.56%

Progress: [50/50]
Current Acc.: 76.00%

3-shot with verification done. Accuracy: 0.760
Progress: [5/50]
Current Acc.: 80.00%

Progress: [10/50]
Current Acc.: 70.00%

Progress: [15/50]
Current Acc

### 보고서 작성하기
#### 아래의 내용이 포함되면 됩니다!

1. Direct Prompting, CoT Prompting, My Prompting을 0 shot, 3 shot, 5 shot 정답률을 표로 보여주세요!
2. CoT Prompting이 Direct Prompting에 비해 왜 좋을 수 있는지에 대해서 서술해주세요!
3. 본인이 작성한 프롬프트 기법이 CoT에 비해서 왜 더 좋을 수 있는지에 대해서 설명해주세요!
4. 최종적으로, `PROMPTING.md`에 보고서를 작성해주세요!